# ========================================
# NEW DATA PIPELINE - START HERE
# ========================================

## Redesigned COMTRADE Data Extraction Pipeline

This pipeline extracts bilateral trade data from the UN COMTRADE API and generates trade matrices in CSV format.

**Features:**
- Configurable country selection, transport mode, and HS commodity codes
- Efficient API calls with proper error handling
- Automatic API key loading from JSON file
- Clear progress tracking and error reporting
- Single CSV matrix output per run

---

## Pipeline Documentation

### How to Use This Pipeline

1. **Configure Settings** (Section 1):
   - Edit `SELECTED_COUNTRIES` list with country names you want to analyze
   - Set `TRANSPORT_MODE` ('ALL', 'SEA_ONLY', 'AIR_ONLY', etc.)
   - Set `HS_CODE` ('TOTAL' or specific code like 'HS72')
   - Set `PERIOD` (year, e.g., '2023')

2. **Create API Key File**:
   - Create a file named `COMTRADE_API_KEY.json` in this directory
   - Format: `{"SUBSCRIPTION_KEY": "your_api_key_here"}`

3. **Run All Cells** in Sections 1-7

4. **Output**:
   - CSV matrix: `./data/new_pipeline/trade_matrix_{mode}_{code}.csv`
   - Metadata JSON: `./data/new_pipeline/metadata_{mode}_{code}.json`

### Trade Matrix Format

- **Rows**: Exporting countries (reporters)
- **Columns**: Importing countries (partners)  
- **Values**: Bilateral export values in USD
- **Index/Columns**: Country names (sorted alphabetically)

### Country Code Resources

Download official country mappings:
- **Reporters**: https://comtradeapi.un.org/files/v1/app/reference/Reporters.json
- **Partner Areas**: https://comtradeapi.un.org/files/v1/app/reference/partnerAreas.json
- **Documentation**: https://unstats.un.org/wiki/display/comtrade/Country+Codes

### Error Handling

- Failed API calls are logged but don't stop the pipeline
- Failed countries are listed in the summary
- Matrix is only generated if at least one country succeeds
- No redundant test API calls (saves quota)

### Notes

- API rate limits: ~0.2s delay between calls to respect limits
- Large country lists may take time (5+ countries per mode)
- 'ALL' transport mode makes 5 API calls per country
- Matrix includes all countries that appear as partners, even if not in your selected list

## Section 1: Configuration & Global Variables

**Adjust these settings before running the pipeline:**

In [ ]:
import json
import pandas as pd
import numpy as np
import comtradeapicall
from typing import Dict, List, Optional
import time
from datetime import datetime
import os

# ============================================
# GLOBAL CONFIGURATION - ADJUST THESE VALUES
# ============================================

# 1. COUNTRY SELECTION
# Option A: List specific country names you want to extract trade data for
# Example: ['USA', 'China', 'Germany', 'France', 'Italy', 'Spain', 'Greece', 'Turkey']
# Option B: Use 'ALL' to extract data for all valid countries (non-expired, non-group countries)
SELECTED_COUNTRIES = "ALL"

# 2. TRANSPORT MODE SELECTION
# Options: 'ALL', 'SEA_ONLY', 'AIR_ONLY', 'ROAD_ONLY', 'RAIL_ONLY', 'PIPELINE_ONLY'
TRANSPORT_MODE = 'ALL'

# 3. HS COMMODITY CODE
# Use 'TOTAL' for all commodities, or specify a 2-digit HS code (e.g., 'HS72' for Iron and Steel)
# Common codes: 'HS10' (Cereals), 'HS27' (Mineral Fuels), 'HS72' (Iron and Steel), 'HS89' (Ships)
HS_CODE = '31'

# 4. TIME PERIOD
# Year for data extraction (e.g., '2023', '2024')
# Note: Most recent year may have incomplete data
PERIOD = '2024'

# 5. OUTPUT DIRECTORY
OUTPUT_DIR = '../../data/all_trade_matrices/'

# 6. API KEY FILE
API_KEY_FILE = '../../COMTRADE_API_KEY.json'

print("✓ Configuration loaded")
if SELECTED_COUNTRIES == 'ALL':
    print(f"  - Countries: ALL (will be determined after loading country codes)")
else:
    print(f"  - Countries: {len(SELECTED_COUNTRIES)} selected")
print(f"  - Transport Mode: {TRANSPORT_MODE}")
print(f"  - HS Code: {HS_CODE}")
print(f"  - Period: {PERIOD}")

## Section 2: Load API Key and Country Codes

In [20]:
# Load API key from JSON file
try:
    with open(API_KEY_FILE, 'r') as f:
        api_config = json.load(f)
        SUBSCRIPTION_KEY = api_config['SUBSCRIPTION_KEY']
    print(f"✓ API key loaded from {API_KEY_FILE}")
except FileNotFoundError:
    print(f"❌ ERROR: {API_KEY_FILE} not found!")
    print(f"   Please create a JSON file with: {{\"SUBSCRIPTION_KEY\": \"your_key_here\"}}")
    raise
except KeyError:
    print(f"❌ ERROR: 'SUBSCRIPTION_KEY' not found in {API_KEY_FILE}")
    raise

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✓ Output directory ready: {OUTPUT_DIR}")

# Fetch country codes from Comtrade API
print("✓ Fetching country codes from Comtrade API...")
try:
    import urllib.request
    
    # Download the reporters reference file
    url = 'https://comtradeapi.un.org/files/v1/app/reference/Reporters.json'
    with urllib.request.urlopen(url) as response:
        reporters_data = json.loads(response.read().decode())
    
    # Create a mapping from country name to reporter code
    # Filter out: groups (isGroup=true) and expired countries (has entryExpiredDate key)
    country_name_to_code = {}
    all_valid_countries = []
    
    for item in reporters_data['results']:
        # Skip if it's a group (like "World", "EU", "ASEAN", etc.)
        if item.get('isGroup', False):
            continue
        
        # Skip if it has an expiry date (historical/dissolved countries)
        if 'entryExpiredDate' in item:
            continue
        
        country_name = item['text']  # Display name
        reporter_code = item['id']  # Numeric code
        country_name_to_code[country_name] = reporter_code
        all_valid_countries.append(country_name)
    
    print(f"✓ Loaded {len(country_name_to_code)} valid country codes")
    print(f"  (Filtered out groups and expired countries)")
    
except Exception as e:
    print(f"❌ ERROR: Failed to fetch country codes: {e}")
    raise

print(f"✓ API ready to use")

✓ API key loaded from COMTRADE_API_KEY.json
✓ Output directory ready: ./data/all_trade_matrices/
✓ Fetching country codes from Comtrade API...
✓ Loaded 219 valid country codes
  (Filtered out groups and expired countries)
✓ API ready to use


## Section 3: Define Helper Functions

In [21]:
def get_transport_mode_codes(mode: str) -> List[str]:
    """
    Convert transport mode string to API mode codes.
    
    Args:
        mode: One of 'ALL', 'SEA_ONLY', 'AIR_ONLY', 'ROAD_ONLY', 'RAIL_ONLY', 'PIPELINE_ONLY'
    
    Returns:
        List of mode codes to query (always a single-element list for efficiency)
    """
    mode_mapping = {
        'SEA_ONLY': ['2100'],
        'AIR_ONLY': ['1000'],
        'ROAD_ONLY': ['3200'],
        'RAIL_ONLY': ['3100'],
        'PIPELINE_ONLY': ['9100'],
        'ALL': [None]  # None = TOTAL (all transport modes in one call)
    }
    
    if mode not in mode_mapping:
        raise ValueError(f"Invalid transport mode: {mode}. Must be one of {list(mode_mapping.keys())}")
    
    return mode_mapping[mode]


def get_hs_code(hs_string: str) -> str:
    """
    Convert HS code string to API format.
    
    Args:
        hs_string: 'TOTAL' or 'HSXX' where XX is a 2-digit code
    
    Returns:
        HS code for API query
    """
    if hs_string == 'TOTAL':
        return 'TOTAL'
    
    if hs_string.startswith('HS') and len(hs_string) == 4:
        return hs_string[2:]  # Extract the 2-digit code
    
    # If it's already a 2-digit code
    if len(hs_string) == 2 and hs_string.isdigit():
        return hs_string
    
    raise ValueError(f"Invalid HS code: {hs_string}. Use 'TOTAL' or 'HSXX' format (e.g., 'HS72')")


def extract_country_trade_data(
    subscription_key: str,
    reporter_code: int,
    reporter_name: str,
    period: str,
    mode_codes: List[str],
    hs_code: str
) -> Optional[pd.DataFrame]:
    """
    Extract trade data for a single country across all transport modes.
    
    Args:
        subscription_key: Comtrade API subscription key
        reporter_code: Numeric reporter code for the country
        reporter_name: Country name (for reference in output)
        period: Year for data extraction
        mode_codes: List of transport mode codes (now always single element)
        hs_code: HS commodity code
    
    Returns:
        DataFrame with aggregated trade data by partner country, or None if extraction fails
    """
    all_data = []
    
    for mode_code in mode_codes:
        try:
            # Query the API using the direct function call with numeric reporter code
            # If mode_code is None, it will use TOTAL (all transport modes aggregated)
            data = comtradeapicall.getFinalData(
                subscription_key,
                typeCode='C',  # Commodities
                freqCode='A',  # Annual
                clCode='HS',   # Harmonized System
                period=period,
                reporterCode=reporter_code,  # Use numeric code instead of name
                cmdCode=hs_code,
                flowCode='X',  # Exports
                partnerCode=None,  # All partners
                partner2Code=None,
                customsCode='C00',  # Default customs
                motCode=mode_code,  # None = TOTAL (all modes), or specific code
                maxRecords=50000,
                format_output='JSON',
                aggregateBy=None,
                breakdownMode='classic',
                countOnly=None,
                includeDesc=True
            )
            
            if data is not None and len(data) > 0:
                all_data.append(data)
            
            # Small delay to respect API rate limits (only one iteration now for 'ALL')
            time.sleep(0.2)
            
        except Exception as e:
            # If one mode fails, continue with others
            continue
    
    # If no data was retrieved for any mode, return None
    if not all_data:
        return None
    
    # Combine all transport modes (though for 'ALL', it's just one call)
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Aggregate by partner country (sum across transport modes and commodity details)
    aggregated = combined_df.groupby('partnerDesc')['primaryValue'].sum().reset_index()
    aggregated.columns = ['partner', 'trade_value']
    
    return aggregated


print("✓ Helper functions defined")

✓ Helper functions defined


## Section 4: Extract Trade Data for All Countries

In [22]:
# Prepare configuration
mode_codes = get_transport_mode_codes(TRANSPORT_MODE)
hs_api_code = get_hs_code(HS_CODE)

# Handle "ALL" option for SELECTED_COUNTRIES
if SELECTED_COUNTRIES == "ALL":
    countries_to_process = sorted(all_valid_countries)
    print(f"✓ 'ALL' mode selected: Will process all {len(countries_to_process)} valid countries")
elif isinstance(SELECTED_COUNTRIES, list):
    countries_to_process = SELECTED_COUNTRIES
else:
    raise ValueError("SELECTED_COUNTRIES must be either 'ALL' or a list of country names")

print(f"\n{'='*60}")
print(f"STARTING DATA EXTRACTION")
print(f"{'='*60}")
print(f"Countries to process: {len(countries_to_process)}")
print(f"Transport modes: {TRANSPORT_MODE}")
print(f"API calls per country: {len(mode_codes)} (Total API calls: {len(countries_to_process) * len(mode_codes)})")
print(f"HS Code: {HS_CODE} (API: {hs_api_code})")
print(f"Period: {PERIOD}")
print(f"{'='*60}\n")

# Validate that all selected countries have codes (only if not "ALL")
if SELECTED_COUNTRIES != "ALL":
    print("Validating country names...")
    invalid_countries = []
    for country in countries_to_process:
        if country not in country_name_to_code:
            invalid_countries.append(country)

    if invalid_countries:
        print(f"\n❌ ERROR: The following countries were not found in Comtrade database:")
        for country in invalid_countries:
            print(f"   - {country}")
        print(f"\nAvailable country names (first 20):")
        for i, name in enumerate(sorted(country_name_to_code.keys())[:20]):
            print(f"   - {name}")
        print(f"   ... and {len(country_name_to_code) - 20} more")
        print(f"\nPlease check the spelling and use exact country names from Comtrade.")
        raise ValueError("Invalid country names in SELECTED_COUNTRIES")

    print(f"✓ All country names validated\n")

# Storage for all country trade data
country_trade_data = {}
failed_countries = []
successful_count = 0

# Extract data for each country
start_time = time.time()

for idx, country_name in enumerate(countries_to_process, 1):
    print(f"[{idx}/{len(countries_to_process)}] Processing: {country_name}...", end=" ")
    
    # Get the country code
    country_code = country_name_to_code[country_name]
    
    try:
        # Extract trade data
        trade_df = extract_country_trade_data(
            subscription_key=SUBSCRIPTION_KEY,
            reporter_code=country_code,
            reporter_name=country_name,
            period=PERIOD,
            mode_codes=mode_codes,
            hs_code=hs_api_code
        )
        
        if trade_df is not None and len(trade_df) > 0:
            # Store the data using country name as key
            country_trade_data[country_name] = trade_df
            successful_count += 1
            print(f"✓ Success ({len(trade_df)} partners)")
        else:
            print(f"❌ No data returned")
            failed_countries.append(country_name)
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        failed_countries.append(country_name)
        continue

elapsed_time = time.time() - start_time

# Summary
print(f"\n{'='*60}")
print(f"EXTRACTION COMPLETE")
print(f"{'='*60}")
print(f"✓ Successful: {successful_count}/{len(countries_to_process)} countries")
print(f"❌ Failed: {len(failed_countries)} countries")
if failed_countries:
    print(f"   Failed countries: {', '.join(failed_countries[:10])}")
    if len(failed_countries) > 10:
        print(f"   ... and {len(failed_countries) - 10} more")
print(f"⏱ Time elapsed: {elapsed_time:.1f} seconds")
print(f"⏱ API calls used: {successful_count * len(mode_codes)}")
print(f"{'='*60}\n")

✓ 'ALL' mode selected: Will process all 219 valid countries

STARTING DATA EXTRACTION
Countries to process: 219
Transport modes: ALL
API calls per country: 1 (Total API calls: 219)
HS Code: 31 (API: 31)
Period: 2024

[1/219] Processing: Afghanistan... ❌ No data returned
[2/219] Processing: Albania... ✓ Success (5 partners)
[3/219] Processing: Algeria... ❌ No data returned
[4/219] Processing: Andorra... ❌ No data returned
[5/219] Processing: Angola... ✓ Success (7 partners)
[6/219] Processing: Anguilla... ❌ No data returned
[7/219] Processing: Antigua and Barbuda... ✓ Success (2 partners)
[8/219] Processing: Argentina... ✓ Success (6 partners)
[9/219] Processing: Armenia... ✓ Success (5 partners)
[10/219] Processing: Aruba... ❌ No data returned
[11/219] Processing: Australia... ✓ Success (62 partners)
[12/219] Processing: Austria... ✓ Success (77 partners)
[13/219] Processing: Azerbaijan... ✓ Success (12 partners)
[14/219] Processing: Bahamas... ❌ No data returned
[15/219] Processing: B

## Section 5: Build Trade Matrix

In [23]:
# Only proceed if we have data
if successful_count == 0:
    print("❌ ERROR: No data was successfully extracted. Cannot build trade matrix.")
    print("   Please check your configuration and API key.")
else:
    print(f"Building trade matrix from {len(countries_to_process)} countries...")
    
    # Use ALL countries from countries_to_process, not just successful ones
    # This ensures countries that appear as importers are included even if their export data failed
    all_matrix_countries = sorted(countries_to_process)
    
    # Add "World" FIRST, then countries (for better overview)
    matrix_countries = ['World'] + all_matrix_countries
    
    print(f"  - Matrix will include: 'World' totals + {len(all_matrix_countries)} countries")
    print(f"  - ({successful_count} countries have export data, others may appear as importers)")
    
    # Initialize the trade matrix with zeros
    trade_matrix = pd.DataFrame(
        0.0,
        index=matrix_countries,
        columns=matrix_countries
    )
    
    # Fill in the trade values
    # Rows = Exporters (reporters)
    # Columns = Importers (partners)
    # Only countries with successful API calls will have non-zero rows
    for exporter, trade_df in country_trade_data.items():
        for _, row in trade_df.iterrows():
            importer = row['partner']
            trade_value = row['trade_value']
            
            # Only add to matrix if the importer is also in our countries_to_process
            if importer in all_matrix_countries:
                trade_matrix.loc[exporter, importer] = trade_value
    
    # Calculate "World" totals (sum across each row and column)
    # For each country's row, the "World" column = sum of that country's exports to all others
    for country in all_matrix_countries:
        trade_matrix.loc[country, 'World'] = trade_matrix.loc[country, all_matrix_countries].sum()
    
    # For each country's column, the "World" row = sum of imports from all others
    for country in all_matrix_countries:
        trade_matrix.loc['World', country] = trade_matrix.loc[all_matrix_countries, country].sum()
    
    # The "World, World" cell = total trade within the selected countries
    trade_matrix.loc['World', 'World'] = trade_matrix.loc[all_matrix_countries, all_matrix_countries].sum().sum()
    
    print(f"✓ Trade matrix built: {trade_matrix.shape}")
    
    # Display basic statistics (excluding World row/column for these stats)
    total_trade = trade_matrix.loc[all_matrix_countries, all_matrix_countries].sum().sum()
    non_zero_cells = (trade_matrix.loc[all_matrix_countries, all_matrix_countries] > 0).sum().sum()
    sparsity = (1 - non_zero_cells / (len(all_matrix_countries) ** 2)) * 100
    
    print(f"  - Total trade value (within selected countries): ${total_trade:,.0f}")
    print(f"  - Non-zero cells: {non_zero_cells:,}")
    print(f"  - Matrix sparsity: {sparsity:.1f}%")
    print(f"  - 'World' row/column added as FIRST row/column")

Building trade matrix from 219 countries...
  - Matrix will include: 'World' totals + 219 countries
  - (114 countries have export data, others may appear as importers)
✓ Trade matrix built: (220, 220)
  - Total trade value (within selected countries): $66,751,968,547
  - Non-zero cells: 4,510
  - Matrix sparsity: 90.6%
  - 'World' row/column added as FIRST row/column


## Section 6: Save Trade Matrix to CSV

In [24]:
if successful_count > 0:
    # Generate filename following the naming convention
    # Format: trade_matrix_{transport_mode}_{commodity}.csv
    
    # Format transport mode for filename
    transport_mode_str = TRANSPORT_MODE.lower().replace('_', '_')
    if TRANSPORT_MODE == 'ALL':
        transport_mode_str = 'all_transport_modes'
    
    # Format HS code for filename
    hs_code_str = HS_CODE if HS_CODE == 'TOTAL' else HS_CODE
    
    # Create filename
    filename = f"trade_matrix_{transport_mode_str}_HS{hs_code_str}.csv"
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    # Save the matrix
    trade_matrix.to_csv(filepath)
    
    print(f"\n{'='*60}")
    print(f"MATRIX SAVED")
    print(f"{'='*60}")
    print(f"✓ Filename: {filename}")
    print(f"✓ Full path: {filepath}")
    print(f"✓ Size: {trade_matrix.shape[0]} x {trade_matrix.shape[1]}")
    print(f"{'='*60}\n")
    
    print(f"🎉 Pipeline complete! Matrix ready for analysis.")


MATRIX SAVED
✓ Filename: trade_matrix_all_transport_modes_HS31.csv
✓ Full path: ./data/all_trade_matrices/trade_matrix_all_transport_modes_HS31.csv
✓ Size: 220 x 220

🎉 Pipeline complete! Matrix ready for analysis.
